In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold # Untuk Cross-Validation
from sklearn.metrics import mean_absolute_error, make_scorer # Untuk metrik kustom
import matplotlib.pyplot as plt
import os

# ==============================================================================
# LANGKAH 1 & 2: MEMUAT DATA (Pastikan sudah memiliki semua fitur yang baru)
# ==============================================================================
# Asumsi path_split_data dan train_set/test_set sudah dimuat dengan semua fitur
path_split_data = r'C:\MyFolder\Git\TA_SpatioTemporal\Data\split_data'
train_set = pd.read_parquet(os.path.join(path_split_data, 'train_set.parquet'))
test_set = pd.read_parquet(os.path.join(path_split_data, 'test_set.parquet'))
print("Data latih dan uji berhasil dimuat.")

TARGET = 'konsumsi_energi'

# ==============================================================================
# LANGKAH 3: PERSIAPAN DATA UNTUK RFECV
# ==============================================================================

# Definisikan semua fitur kandidat yang Anda buat di script feature_engineering
# Ini adalah semua fitur yang mungkin Anda ingin RFECV pertimbangkan
# Salin daftar ini dari LANGKAH 3 di notebook 01_LightGBM_Model.ipynb yang sudah diperbarui
all_candidate_features = [
    'jam', 'hari_minggu', 'hari_bulan', 'minggu_tahun', 'bulan', 'tahun',
    'apakah_akhir_pekan', 'apakah_jam_kerja', 'meter_id',
    'is_kelas', 'is_kantor', 'is_penelitian',
    'konsumsi_energi_lag_1_jam', 'avg_temp_previous_hour_lag_1_jam',
    'konsumsi_energi_lag_24_jam', 'avg_temp_previous_hour_lag_24_jam',
    'konsumsi_energi_lag_168_jam', 'avg_temp_previous_hour_lag_168_jam',
    'konsumsi_energi_lag_720_jam', 'avg_temp_previous_hour_lag_720_jam'
]

# Tambahkan fitur agregasi yang di-lag. Anda bisa membuat loop di sini atau menyalinnya.
# Contoh membuat ulang daftar fitur agregasi yang di-lag:
lag_periods = [1, 24, 168, 720]
rolling_windows = [24, 168, 720]
features_lagged_agg = []
for lag in lag_periods:
    for rolling_window in rolling_windows:
        features_lagged_agg.append(f'konsumsi_energi_rolling_mean_{rolling_window}h_at_lag_{lag}_jam')
        features_lagged_agg.append(f'avg_temp_previous_hour_rolling_mean_{rolling_window}h_at_lag_{lag}_jam')

all_candidate_features.extend(features_lagged_agg)

# Filter fitur yang benar-benar ada di train_set (mencegah error jika ada fitur yang tidak ada)
final_features_for_rfe = [f for f in all_candidate_features if f in train_set.columns and f != TARGET]

X_rfe = train_set[final_features_for_rfe].copy() # Tambahkan .copy() untuk menghindari SettingWithCopyWarning
y_rfe = train_set[TARGET]

# ==============================================================================
# PERBAIKAN: Lakukan One-Hot Encoding untuk fitur kategorikal
# RFECV memerlukan input numerik penuh.
# ==============================================================================
# Definisikan kolom yang akan di-one-hot encode
# 'meter_id' adalah kolom string yang menyebabkan error 'BSC A'
# Fitur biner (apakah_akhir_pekan, apakah_jam_kerja, is_kelas, is_kantor, is_penelitian)
# seharusnya sudah numerik (0/1) dari script feature_engineering, jadi tidak perlu di-one-hot lagi.
# Fitur waktu seperti hari_minggu, bulan, minggu_tahun, tahun bisa dipertimbangkan untuk OHE juga
# jika Anda tidak ingin LightGBM memperlakukannya sebagai numerik ordinal.
# Untuk RFECV, lebih aman untuk OHE semuanya yang non-numerik intrinsik.
cols_to_onehot_encode = ['meter_id'] # Fokus utama pada 'meter_id' yang menyebabkan error

# Jika ada fitur kategorikal lain yang Anda inginkan di-one-hot encode untuk RFECV,
# tambahkan ke daftar ini. Contoh:
# cols_to_onehot_encode.extend(['hari_minggu', 'bulan', 'minggu_tahun', 'tahun'])

# Lakukan one-hot encoding
X_rfe = pd.get_dummies(X_rfe, columns=cols_to_onehot_encode, drop_first=True) # drop_first=True untuk menghindari multicollinearity

# Sekarang, `X_rfe` hanya berisi fitur numerik (termasuk hasil one-hot encoding)
print(f"\nTotal fitur setelah One-Hot Encoding: {len(X_rfe.columns)}")
print(f"Contoh fitur setelah One-Hot Encoding: {X_rfe.columns.tolist()[:5]}")


# ==============================================================================
# LANGKAH 4: IMPLEMENTASI RFECV DENGAN LIGHTGBM
# ==============================================================================

# Inisialisasi LightGBM Regressor sebagai estimator
# Menggunakan parameter yang sama dengan model utama Anda
# Karena kita sudah melakukan One-Hot Encoding, kita tidak perlu lagi
# meneruskan categorical_feature secara eksplisit ke estimator di sini.
# LightGBM akan melihatnya sebagai kolom numerik (0/1).
estimator = lgb.LGBMRegressor(
    objective='regression_l1', # Objective yang digunakan (misalnya MAE)
    n_estimators=500, # Mengurangi estimator untuk RFECV agar lebih cepat
    learning_rate=0.05,
    num_leaves=31,
    n_jobs=-1,
    seed=42,
    # categorical_feature tidak lagi diperlukan di sini karena sudah di-one-hot encode
)

# Definisikan metrik scoring untuk RFECV
# Untuk regresi, 'neg_mean_squared_error' atau 'neg_mean_absolute_error'
# make_scorer diperlukan jika Anda ingin menggunakan MAE sebagai scoring
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False) # greater_is_better=False untuk error metrics

# Inisialisasi RFECV
# cv: jumlah fold untuk cross-validation
# scoring: metrik yang akan dioptimalkan
# step: jumlah fitur yang dieliminasi di setiap iterasi (bisa int atau float 0-1)
# min_features_to_select: jumlah minimum fitur yang harus dipertahankan
rfecv = RFECV(
    estimator=estimator,
    step=1, # Menghapus 1 fitur di setiap iterasi
    cv=KFold(n_splits=3, shuffle=True, random_state=42), # Menggunakan KFold untuk CV
    scoring=mae_scorer, # Menggunakan MAE sebagai metrik scoring
    min_features_to_select=1 # Jangan sampai semua fitur tereliminasi
)

print("\nMulai menjalankan RFECV (ini mungkin membutuhkan waktu)...")
rfecv.fit(X_rfe, y_rfe)
print("RFECV selesai.")

# ==============================================================================
# LANGKAH 5: ANALISIS HASIL RFECV
# ==============================================================================
print(f"\nJumlah fitur optimal yang ditemukan: {rfecv.n_features_}")
# PERBAIKAN: Mengganti 'grid_scores_' dengan 'cv_results_['mean_test_score']'
print(f"Mean cross-validation score untuk jumlah fitur optimal: {-rfecv.cv_results_['mean_test_score'][rfecv.n_features_ - 1]:.4f} (MAE)")

# Dapatkan nama-nama fitur yang terpilih
selected_features_by_rfe = X_rfe.columns[rfecv.support_].tolist()
print(f"\nFitur-fitur yang terpilih oleh RFECV ({len(selected_features_by_rfe)} fitur):")
print(selected_features_by_rfe)

# Plot performa model vs. jumlah fitur
plt.figure(figsize=(12, 6))
plt.xlabel("Jumlah Fitur yang Dipilih")
plt.ylabel("Skor MAE (negatif)") # Perhatikan bahwa RFECV mengoptimalkan neg_mean_absolute_error
# PERBAIKAN: Mengganti 'grid_scores_' dengan 'cv_results_['mean_test_score']'
plt.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1), rfecv.cv_results_['mean_test_score'])
plt.grid(True)
plt.title("RFECV: Performa Model vs. Jumlah Fitur")
plt.show()

# ==============================================================================
# LANGKAH 6: MEMPERBARUI SELECTED_FEATURES DI NOTEBOOK UTAMA
# ==============================================================================
# Sekarang, Anda bisa menggunakan `selected_features_by_rfe` ini
# untuk mengatur SELECTED_FEATURES di LANGKAH 4 pada notebook 01_LightGBM_Model.ipynb.
# Anda bisa membuat skenario baru di sana, misalnya PILIHAN_SKENARIO = 6,
# dan set SELECTED_FEATURES = selected_features_by_rfe
# Atau, Anda dapat secara manually memasukkan fitur-fitur ini ke dalam skenario yang sudah ada.

# Contoh bagaimana Anda bisa mengatur SELECTED_FEATURES di notebook utama:
# PILIHAN_SKENARIO = 6 # Misalnya, skenario baru untuk fitur hasil RFECV
# if PILIHAN_SKENARIO == 6:
#    SELECTED_FEATURES = selected_features_by_rfe
# ... (lanjutkan dengan proses pelatihan model di notebook utama)

print("\n--- Saran Selanjutnya ---")
print("Salin daftar 'Fitur-fitur yang terpilih oleh RFECV' di atas.")
print("Kemudian, buka kembali notebook '01_LightGBM_Model.ipynb'.")
print("Anda bisa membuat skenario baru di LANGKAH 4, misalnya PILIHAN_SKENARIO = 6.")
print("Dalam skenario ini, atur `SELECTED_FEATURES = [daftar_fitur_yang_disalin_di_atas]`.")
print("Jalankan ulang notebook utama untuk melihat performa model dengan fitur yang dioptimalkan oleh RFECV.")


Data latih dan uji berhasil dimuat.

Total fitur setelah One-Hot Encoding: 82
Contoh fitur setelah One-Hot Encoding: ['jam', 'hari_minggu', 'hari_bulan', 'minggu_tahun', 'bulan']

Mulai menjalankan RFECV (ini mungkin membutuhkan waktu)...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011793 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8365
[LightGBM] [Info] Number of data points in the train set: 175253, number of used features: 82
[LightGBM] [Info] Start training from score 13.288600


c:\Users\Danu Hakim\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021395 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8363
[LightGBM] [Info] Number of data points in the train set: 175253, number of used features: 81
[LightGBM] [Info] Start training from score 13.288600


c:\Users\Danu Hakim\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007838 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8360
[LightGBM] [Info] Number of data points in the train set: 175253, number of used features: 80
[LightGBM] [Info] Start training from score 13.288600


...

c:\Users\Danu Hakim\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000441 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 175254, number of used features: 1
[LightGBM] [Info] Start training from score 13.261745


c:\Users\Danu Hakim\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014097 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8365
[LightGBM] [Info] Number of data points in the train set: 262880, number of used features: 82
[LightGBM] [Info] Start training from score 13.289391
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012173 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8363
[LightGBM] [Info] Number of data points in the train set: 262880, number of used features: 81
[LightGBM] [Info] Start training from score 13.289391
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.042035 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Tota

AttributeError: 'RFECV' object has no attribute 'grid_scores_'